In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/heart-diseases/heart (1).csv


In [2]:
df = pd.read_csv("/kaggle/input/heart-diseases/heart (1).csv")
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
X = df.drop(columns=['target'])
y = df['target']

In [4]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=8)

In [5]:
X_train.shape

(242, 13)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

Stacking

In [7]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
    ('knn', KNeighborsClassifier(n_neighbors=10)),
    ('gbdt',GradientBoostingClassifier())
]

In [8]:
from sklearn.ensemble import StackingClassifier
#it uses K-fold prediction internally 
clf = StackingClassifier(
    estimators = estimators,
    final_estimator = LogisticRegression(),
    cv = 10 # k 
)

In [9]:
clf.fit(X_train,y_train)

StackingClassifier(cv=10,
                   estimators=[('rf',
                                RandomForestClassifier(n_estimators=10,
                                                       random_state=42)),
                               ('knn', KNeighborsClassifier(n_neighbors=10)),
                               ('gbdt', GradientBoostingClassifier())],
                   final_estimator=LogisticRegression())

In [10]:
y_pred = clf.predict(X_test)

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.8688524590163934

k-fold manually 

In [12]:
final_estimator = LogisticRegression()

In [13]:
estimators

[('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
 ('knn', KNeighborsClassifier(n_neighbors=10)),
 ('gbdt', GradientBoostingClassifier())]

In [14]:
from sklearn.model_selection import KFold
import numpy as np

kf = KFold(n_splits=5, shuffle=True, random_state=42)
#oof -> out of fold
oof_preds = np.zeros((len(X), len(estimators)))  # training level-2 data
test_preds = np.zeros((len(X), len(estimators))) # for final prediction on X

for i, (name, model) in enumerate(estimators):
    fold_test_pred = np.zeros(len(X))

    for train_idx, val_idx in kf.split(X):
        X_train = X.iloc[train_idx]
        X_val   = X.iloc[val_idx]
        y_train = y.iloc[train_idx]

        model.fit(X_train, y_train)

        # out-of-fold predictions
        oof_preds[val_idx, i] = model.predict_proba(X_val)[:, 1]

        # prediction on full data for later final blending
        fold_test_pred[val_idx] = model.predict_proba(X_val)[:, 1]

    test_preds[:, i] = fold_test_pred

# train meta-model
final_estimator.fit(oof_preds, y)

# final stacked prediction
y_pred = final_estimator.predict(test_preds)

print("Stacking Accuracy:", accuracy_score(y, y_pred))


Stacking Accuracy: 0.7887788778877888


Blending

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_hold, y_train, y_hold = train_test_split(
    X, y, test_size=0.2, random_state=42
)

blend_preds = []

for name, model in estimators:
    model.fit(X_train, y_train)
    blend_preds.append(model.predict_proba(X_hold)[:, 1]) #predict_proba return probablistic rpedictions

# stack column-wise for meta model
blend_X = np.column_stack(blend_preds)

# train meta model
final_estimator.fit(blend_X, y_hold)

# predict on holdout
y_pred_blend = final_estimator.predict(blend_X)

print("Blending Accuracy:", accuracy_score(y_hold, y_pred_blend))


Blending Accuracy: 0.8688524590163934
